In [ ]:
#!/usr/bin/env python
# coding: utf-8

# read cif file and make super cell

# In[10]:


#!/usr/bin/env python
# coding: utf-8

# read cif file and make super cell

# In[48]:

from scipy import special
import ase
from ase.io import read
import numpy as np
import pandas as pd
from scipy.spatial import distance
#list of effective neighbor atoms based on the wall atom
input_addr,output_addr="1_charged.cif","1.txt"
####generate supercell####
# Read in the CIF file
print("reading "+input_addr) 
atoms = read(input_addr)

    
cif=np.loadtxt(input_addr, delimiter='|',dtype=str)
contains_ab = np.vectorize(lambda x: '_atom_site_charge' in x)
valid_content=cif[np.where(contains_ab(cif))[0][0]+1:]
split_array=np.char.split(valid_content)
charges_origin=np.array([list(item) for item in split_array])[:,-1].astype(float).tolist()    
atoms.set_initial_charges(charges=charges_origin)
#print(len(atoms.get_initial_charges()))
cutoff=12
#make supercell to meet LJ cut-off reequirement
xlength=np.linalg.norm(atoms.get_cell()[0])
ylength=np.linalg.norm(atoms.get_cell()[1])
zlength=np.linalg.norm(atoms.get_cell()[2])
if cutoff*2%xlength==0: mul_x=int(cutoff*2/xlength) 
else: mul_x=int(cutoff*2/xlength)+1
if cutoff*2%ylength==0: mul_y=int(cutoff*2/ylength) 
else: mul_y=int(cutoff*2/ylength)+1
if cutoff*2%zlength==0: mul_z=int(cutoff*2/zlength) 
else: mul_z=int(cutoff*2/zlength)+1
    
    
#print(mul_x,mul_y,mul_z)
supercell = ase.build.make_supercell(atoms, [[mul_x, 0, 0], [0, mul_y, 0], [0, 0, mul_z]])

#supercell.positions

symbols=supercell.get_chemical_symbols()
positions=supercell.get_positions()
scaled_positions=supercell.get_scaled_positions()
unitcell=np.array(supercell.get_cell())
inv_unitcell=np.linalg.inv(unitcell)
range_supercell=range(len(supercell))
#all_distances=supercell.get_all_distances(mic=True)
charges=supercell.get_initial_charges()


# In[ ]:


#print(unitcell)

df = pd.read_csv("LJ.csv")
arr= df.to_numpy()
epsilon_dict = dict(zip(arr[:,0], arr[:,1]))
sigma_dict = dict(zip(arr[:,0], arr[:,2]))
epsilon_arr=np.array([epsilon_dict[element] for element in symbols])
sigma_arr=np.array([sigma_dict[element] for element in symbols])

diameter_dict=dict(zip(arr[:,0], arr[:,2]))
diameter_dict['H'] = 1.0 #Hydrogen diameter correction
diameter_arr=np.array([diameter_dict[element] for element in symbols])
#supercell.write('mystructure_supercell.cif')


# detect surface

# In[24]:


#read force field

epsilon_gas,sigma_gas=79.000000,3.050000
r_gas=sigma_gas/2
r_max=4.2/2
q_gas=-0.350
#q_gas=0
# In[25]:


#a function to generate random points on the nearby surface of an atom
import random
import math
import numpy as np

def set_probe(r_gas,r_wall,n):
    # Generate random spherical coordinates
    theta = np.random.uniform(0, 2*math.pi,n)
    phi = np.random.uniform(0, math.pi,n)

    # Convert spherical coordinates to Cartesian coordinates
    x = np.sin(phi) * np.cos(theta)
    y = np.sin(phi) * np.sin(theta)
    z = np.cos(phi)

    r=r_gas+r_wall

    x=x*r
    y=y*r
    z=z*r
    
    return np.array([x,y,z]).T


# In[36]:


#generate surface for each atom
surface_list=[]
weighted_surface=4*np.pi*(r_gas+1.5)*(r_gas+1.5) #used for unifying surface density
for i in range_supercell:
#for i in range(301,302):
    atom=supercell[i]
    r_wall=diameter_dict[atom.symbol]/2
    n_point=int(500/weighted_surface*(4*np.pi*(r_gas+r_wall)*(r_gas+r_wall)))
    #generate sphere for each atom
    sphere=atom.position+set_probe(r_gas,r_wall,n=n_point)
    r_sphere=r_gas+r_wall
    area_point=4*np.pi*r_sphere*r_sphere/n_point
    sphere_scaled_positions=np.dot(sphere,inv_unitcell)
    #first set a neighbor list based on the wall atom, which is used for identifing surface
    #all_distances_i=all_distances[i]
    all_distances_i=supercell.get_distances(i,range_supercell,mic=True)
    r_neighbor=r_wall+r_gas*2+r_max
    sort_neighbor=np.logical_and(all_distances_i>0.0,all_distances_i<r_neighbor)
    neighbor_wallatoms=supercell[sort_neighbor]
    neighbor_diameter=diameter_arr[sort_neighbor]
    neighbor_scaled_positions=neighbor_wallatoms.get_scaled_positions()
    neighbor_symbols=neighbor_wallatoms.get_chemical_symbols()

    #identify the points on the sphere belonging to the surface, and gether surfaces together
    for j in range(sphere.shape[0]):
        #make sphere point at the center of the box to reduce distance calculatio cost due to periodic boundary condition
        neighbor_scaled_positions_opt=neighbor_scaled_positions+np.round(sphere_scaled_positions[j]-neighbor_scaled_positions)
        neighbor_positions_opt=np.dot(neighbor_scaled_positions_opt,unitcell)
        neighbor_distances=np.linalg.norm(sphere[j]-neighbor_positions_opt,axis=1)
        #surface points should be away enough from the neighbor wall atoms(> r_gas+r_wall)
        neighbor_min_dist=neighbor_diameter/2+r_gas
        if np.all(neighbor_distances-neighbor_min_dist>0):
            surfacepoint=sphere[j].tolist()
            surfacepoint.append(area_point)        
            surface_list.append(surfacepoint)
# In[37]:

volume=supercell.get_volume() #Angstrom^2/u.c.
mass=np.sum(supercell.get_masses())/(6.02214076e23) #g/u.c.
density=mass/volume*1e24 #g/cm^3
np.savetxt('density.txt', [density])

if surface_list==[]:
    np.savetxt('surface_features.txt', [0.0]*801)


# In[ ]:


# energy calculation

# In[38]:

In [14]:
U_list=[]
Ulj_list=[]
Uq_list=[]
surface_arr=np.array(surface_list)
surface_positions=surface_arr[:,0:3]
surface_scaled_positions=np.dot(surface_positions,inv_unitcell)
for k in range(len(surface_list)):

    point_position=surface_positions[k]
    point_scaled_position=surface_scaled_positions[k]
    #make sphere point at the center of the box to reduce distance calculatio cost due to periodic boundary condition
    scaled_positions_opt=scaled_positions+np.round(point_scaled_position-scaled_positions)
    positions_opt=np.dot(scaled_positions_opt,unitcell)
    point_distances=np.linalg.norm(point_position-positions_opt,axis=1)


    #LJ potential calculation
    lj_arr=np.vstack((point_distances,epsilon_arr,sigma_arr,charges)).T
    lj_arr=lj_arr[lj_arr[:,0]<cutoff]
    
    epsilon_mixingrule=np.sqrt(lj_arr[:,1]*epsilon_gas)
    sigma_mixingrule=(lj_arr[:,2]+sigma_gas)/2    
    
    distance=lj_arr[:,0]
    epsilon=epsilon_mixingrule
    sigma=sigma_mixingrule
    q_atom=lj_arr[:,3]
    U_lj=np.sum(4*epsilon*(np.power(sigma/distance,12)-np.power(sigma/distance,6)))*0.0083115 #0.0083115Kelvin to kJ/mol
    qq=q_gas*q_atom
    U_q=np.sum(1e-3*6.022e23*9e19*1.602176634e-19*1.602176634e-19*(qq/distance*(1-special.erf(0.15*distance))-qq/cutoff*(1-special.erf(0.15*cutoff))))
    U_list.append(U_lj+U_q)
    Ulj_list.append(U_lj)
    Uq_list.append(U_q)
#U_list


# In[ ]:


# In[39]:


final_arr=np.vstack((surface_arr.T,Ulj_list))
final_arr=np.vstack((final_arr,Uq_list)).T

#final output [x,y,z,area_density/Angstrom^2,potential/Kelvin]
np.savetxt(output_addr,final_arr,delimiter=",")


# In[ ]:


# In[52]:




#surface area
accesable_surface_area=sum(np.array(surface_list)[:,3])

# In[41]:


#histogram
Ulj_arr=np.array(Ulj_list)
Ulj_arr = np.where(Ulj_arr > 0, 0.0, Ulj_arr)
Ulj_arr = np.where(Ulj_arr <= -40, -39.9, Ulj_arr)
Uq_arr=np.array(Uq_list)
Uq_arr = np.where(Uq_arr > 40, 39.9, Uq_arr)
Uq_arr = np.where(Uq_arr <= -40, -39.9, Uq_arr)
Uljq_arr=np.vstack((Ulj_arr,Uq_arr))
Uanddensity_arr=np.vstack((Uljq_arr,surface_arr[:,3])).T


# In[ ]:


bin_list=[]
for m in np.linspace(-35, 0, num=8):
    for q in np.linspace(-35, 40, num=16):
        bin_Uanddensity=Uanddensity_arr[np.logical_and(Uanddensity_arr[:,0]>(m-5.0),Uanddensity_arr[:,0]<=m)]
        bin_Uanddensity=bin_Uanddensity[np.logical_and(bin_Uanddensity[:,1]>(q-5.0),bin_Uanddensity[:,1]<=q)]
        bin_U=np.sum(bin_Uanddensity[:,2])
        bin_list.append(bin_U)
result_hist=[accesable_surface_area/volume/density*10000]+(bin_list/accesable_surface_area).tolist()
#np.savetxt('hist.txt', result_hist)

reading 1_charged.cif
110


In [18]:
#histogram
Ulj_arr=np.array(Ulj_list)
Ulj_arr = np.where(Ulj_arr > 0, 0.0, Ulj_arr)
Ulj_arr = np.where(Ulj_arr <= -40, -39.9, Ulj_arr)
Uq_arr=np.array(Uq_list)
Uq_arr = np.where(Uq_arr > 20, 20.0, Uq_arr)
Uq_arr = np.where(Uq_arr <= -40, -39.9, Uq_arr)
Uljq_arr=np.vstack((Ulj_arr,Uq_arr))
Uanddensity_arr=np.vstack((Uljq_arr,surface_arr[:,3])).T
n_bin=12
rdf_list=[]
final_arr_2=np.vstack((surface_arr.T,Ulj_arr))
final_arr_2=np.vstack((final_arr_2,Uq_arr)).T


# In[22]:


for i in list(range(-35,1,5)):
    for j in list(range(-20,21,20)):
        pcl=final_arr_2[final_arr_2[:,4]<i]
        pcl=pcl[pcl[:,5]<j]
        pcl=pcl[pcl[:,5]>(j-20.0)][:,0:3]
        pcl_scaled=np.dot(pcl,inv_unitcell)
        bin_arr_final=np.array([0.0]*n_bin)
        num_point=len(pcl_scaled[:,0])
        bin_radius_arr=np.linspace(12.0/n_bin,12.0,num=n_bin)
        for m in range(num_point):
        #m=0
            pcl_scaled_opt=pcl_scaled+np.round(pcl_scaled[m]-pcl_scaled)
            pcl_opt=np.dot(pcl_scaled_opt,unitcell)
            neighbor_distances=np.linalg.norm(pcl[m]-pcl_opt,axis=1)
            bin_arr=np.array([])
            for b in bin_radius_arr:
                bin_arr=np.append(bin_arr,len(neighbor_distances[np.logical_and(neighbor_distances<b,neighbor_distances>=b-12.0/n_bin)]))
            bin_arr_final=bin_arr_final+bin_arr
        #bin_arr_final
        volume_weight=4/3*np.pi*(np.power(bin_radius_arr,3)-np.power(bin_radius_arr-12.0/n_bin,3))
        rdf=bin_arr_final/volume_weight/num_point
        rdf_list=rdf_list+rdf.tolist()
rdf1_list = np.nan_to_num(rdf_list, nan=0.0)
    #rdf


C:\Users\dengz\AppData\Local\Temp\ipykernel_8392\718076939.py:39: RuntimeWarning: invalid value encountered in divide
  rdf=bin_arr_final/volume_weight/num_point


In [25]:
Ulj_arr=np.array(Ulj_list)
Ulj_arr = np.where(Ulj_arr > 0, 0.0, Ulj_arr)
Ulj_arr = np.where(Ulj_arr <= -40, -39.9, Ulj_arr)
Uq_arr=np.array(Uq_list)
Uq_arr = np.where(Uq_arr > 40, 39.9, Uq_arr)
Uq_arr = np.where(Uq_arr <= -40, -39.9, Uq_arr)
Uljq_arr=np.vstack((Ulj_arr,Uq_arr))
Uanddensity_arr=np.vstack((Uljq_arr,surface_arr[:,3])).T

n_bin=6
rdf_list=[]
final_arr_2=np.vstack((surface_arr.T,Ulj_arr))
final_arr_2=np.vstack((final_arr_2,Uq_arr)).T


# In[9]:


for i in list(range(-30,1,10)):
#for i in list(range(-0,1,10)):
    for j in list(range(1,5)):
        #print(i,j)
        pcl0=final_arr_2[final_arr_2[:,4]<i]
        pcl=pcl0[pcl0[:,5]<-40.0+10*j][:,0:3]
        pcl_scaled=np.dot(pcl,inv_unitcell)
        pcl2=pcl0[pcl0[:,5]>40.0-10*j][:,0:3]
        pcl2_scaled=np.dot(pcl2,inv_unitcell)
        bin_arr_final=np.array([0.0]*n_bin)
        num_point=len(pcl_scaled[:,0])
        bin_radius_arr=np.linspace(6.0/n_bin,6.0,num=n_bin)
        for m in range(num_point):
        #m=0
            pcl_scaled_opt=pcl2_scaled+np.round(pcl_scaled[m]-pcl2_scaled)
            pcl_opt=np.dot(pcl_scaled_opt,unitcell)
            neighbor_distances=np.linalg.norm(pcl[m]-pcl_opt,axis=1)
            bin_arr=np.array([])
            for b in bin_radius_arr:
                bin_arr=np.append(bin_arr,len(neighbor_distances[np.logical_and(neighbor_distances<b,neighbor_distances>=b-6.0/n_bin)]))
            bin_arr_final=bin_arr_final+bin_arr
        #bin_arr_final
        volume_weight=4/3*np.pi*(np.power(bin_radius_arr,3)-np.power(bin_radius_arr-6.0/n_bin,3))
        rdf=bin_arr_final/volume_weight/num_point
        rdf_list=rdf_list+rdf.tolist()
    #rdf
rdf2_list = np.nan_to_num(rdf_list, nan=0.0)



C:\Users\dengz\AppData\Local\Temp\ipykernel_8392\260274951.py:42: RuntimeWarning: invalid value encountered in divide
  rdf=bin_arr_final/volume_weight/num_point


In [26]:
Ulj_arr=np.array(Ulj_list)
Ulj_arr = np.where(Ulj_arr > 0, 0.0, Ulj_arr)
Ulj_arr = np.where(Ulj_arr <= -40, -39.9, Ulj_arr)
Uq_arr=np.array(Uq_list)
Uq_arr = np.where(Uq_arr > 40, 39.9, Uq_arr)
Uq_arr = np.where(Uq_arr <= -20, -19.9, Uq_arr)
Uljq_arr=np.vstack((Ulj_arr,Uq_arr))
Uanddensity_arr=np.vstack((Uljq_arr,surface_arr[:,3])).T
n_bin=12
rdf_list=[]
final_arr_2=np.vstack((surface_arr.T,Ulj_arr))
final_arr_2=np.vstack((final_arr_2,Uq_arr)).T


# In[18]:


for i in list(range(-35,1,5)):
    for j in list(range(0,41,20)):
        pcl=final_arr_2[final_arr_2[:,4]<i]
        pcl=pcl[pcl[:,5]<j]
        pcl=pcl[pcl[:,5]>(j-20.0)][:,0:3]
        pcl_scaled=np.dot(pcl,inv_unitcell)
        bin_arr_final=np.array([0.0]*n_bin)
        num_point=len(pcl_scaled[:,0])
        bin_radius_arr=np.linspace(12.0/n_bin,12.0,num=n_bin)
        for m in range(num_point):
        #m=0
            pcl_scaled_opt=pcl_scaled+np.round(pcl_scaled[m]-pcl_scaled)
            pcl_opt=np.dot(pcl_scaled_opt,unitcell)
            neighbor_distances=np.linalg.norm(pcl[m]-pcl_opt,axis=1)
            bin_arr=np.array([])
            for b in bin_radius_arr:
                bin_arr=np.append(bin_arr,len(neighbor_distances[np.logical_and(neighbor_distances<b,neighbor_distances>=b-12.0/n_bin)]))
            bin_arr_final=bin_arr_final+bin_arr
        #bin_arr_final
        volume_weight=4/3*np.pi*(np.power(bin_radius_arr,3)-np.power(bin_radius_arr-12.0/n_bin,3))
        rdf=bin_arr_final/volume_weight/num_point
        rdf_list=rdf_list+rdf.tolist()
rdf3_list = np.nan_to_num(rdf_list, nan=0.0)


C:\Users\dengz\AppData\Local\Temp\ipykernel_8392\3905191633.py:38: RuntimeWarning: invalid value encountered in divide
  rdf=bin_arr_final/volume_weight/num_point


In [27]:
np.savetxt('surface_features.txt', result_hist+rdf1_list.tolist()+rdf2_list.tolist()+rdf3_list.tolist())